In [ ]:
# fill in 10RA and seasonal stats

In [ ]:
import pandas as pd
data = pd.read_csv("preprocess_2.csv")
data['date'] = pd.to_datetime(data['date'])
data.head()

,id,home_team_abbr,away_team_abbr,date,is_night_game,home_team_win,home_pitcher,away_pitcher,home_team_rest,away_team_rest,...,away_pitcher_H_batters_faced_skew,away_pitcher_BB_batters_faced_mean,away_pitcher_BB_batters_faced_std,away_pitcher_BB_batters_faced_skew,away_pitcher_leverage_index_avg_mean,away_pitcher_leverage_index_avg_std,away_pitcher_leverage_index_avg_skew,away_pitcher_wpa_def_mean,away_pitcher_wpa_def_std,away_pitcher_wpa_def_skew
0,0,11,12,2021-05-16,False,True,366.0,120.0,1.0,1.0,...,1.133350,-0.974559,-0.900633,-1.093425,0.896974,-0.611051,-0.398111,0.949021,1.007072,0.340438
1,1,25,8,2019-05-04,True,False,613.0,637.0,2.0,NaN,...,-0.044641,-0.878649,-1.079528,-1.719608,0.050448,-0.851738,-0.202878,0.489511,-0.876286,1.416154
2,2,25,10,2019-06-10,True,True,352.0,759.0,1.0,1.0,...,-0.100180,-1.702937,-0.867762,1.992552,-0.404961,-0.132717,-0.106344,2.481020,-0.200110,-0.026083
3,3,0,3,2018-06-26,True,True,186.0,360.0,1.0,1.0,...,-1.385079,-1.549095,-1.008470,0.116080,-1.236753,-0.119898,0.005985,1.646317,-0.764309,NaN
4,4,21,1,2016-07-05,True,False,809.0,30.0,1.0,1.0,...,0.356122,0.663967,-0.123547,0.361822,-0.035276,-0.285671,-2.563819,0.527432,-0.911987,-1.109533


In [2]:
from tqdm import trange

prefix = ["home_", "away_"]
Rolling_Average = [
"batting_batting_avg_10RA"
,"batting_onbase_perc_10RA"
,"batting_onbase_plus_slugging_10RA"
,"batting_leverage_index_avg_10RA"
,"batting_RBI_10RA"
,"pitching_earned_run_avg_10RA"
,"pitching_SO_batters_faced_10RA"
,"pitching_H_batters_faced_10RA"
,"pitching_BB_batters_faced_10RA"]

for abbr in trange(30):
    for season in range(2016, 2024):
        for pre in prefix:
                for RA in Rolling_Average:
                    RA = pre + RA
                    fliter1 = data[pre + "team_abbr"] == abbr
                    filter2 = data["season"] == season
                    temp = data[(fliter1 & filter2)][["date", RA]]
                    temp = temp.sort_values("date").reset_index()
                    
                    j = 0
                    for j in range(temp.shape[0]):
                        if not pd.isna(temp[RA][j]):
                            break
                    for i in range(j):
                        data.loc[temp["index"][i], RA] = temp[RA][j]


                    for i in range(j, temp.shape[0]):
                        if pd.isna(temp[RA][i]):
                            if i == temp.shape[0] - 1:
                                data.loc[temp["index"][i], RA] = data.loc[temp["index"][i-1], RA]
                            elif pd.isna(temp[RA][i+1]):
                                data.loc[temp["index"][i], RA] = data.loc[temp["index"][i-1], RA]
                            else:
                                data.loc[temp["index"][i], RA] = (data.loc[temp["index"][i-1], RA] + data.loc[temp["index"][i+1], RA]) / 2

100%|██████████| 30/30 [00:11<00:00,  2.58it/s]


In [3]:
from tqdm import trange

prefix = ["home_", "away_"]
suffix = ["_mean", "_std", "_skew"]
Team_Seasonal = [
"team_errors"
,"team_spread"
,"team_wins"
,"batting_batting_avg"
,"batting_onbase_perc"
,"batting_onbase_plus_slugging"
,"batting_leverage_index_avg"
,"batting_wpa_bat"
,"batting_RBI"
,"pitching_earned_run_avg"
,"pitching_SO_batters_faced"
,"pitching_H_batters_faced"
,"pitching_BB_batters_faced"
,"pitching_leverage_index_avg"
,"pitching_wpa_def"]

for abbr in trange(30):
    for season in range(2016, 2024):
        for pre in prefix:
            for suf in suffix:
                for TS in Team_Seasonal:
                    TS = pre + TS + suf
                    fliter1 = data[pre + "team_abbr"] == abbr
                    filter2 = data["season"] == season
                    temp = data[(fliter1 & filter2)][["date", TS]]
                    temp = temp.sort_values("date").reset_index()
                    
                    j = 0
                    for j in range(temp.shape[0]):
                        if not pd.isna(temp[TS][j]):
                            break
                    for i in range(j):
                        data.loc[temp["index"][i], TS] = temp[TS][j]


                    for i in range(j, temp.shape[0]):
                        if pd.isna(temp[TS][i]):
                            if i == temp.shape[0] - 1:
                                data.loc[temp["index"][i], TS] = data.loc[temp["index"][i-1], TS]
                            elif pd.isna(temp[TS][i+1]):
                                data.loc[temp["index"][i], TS] = data.loc[temp["index"][i-1], TS]
                            else:
                                data.loc[temp["index"][i], TS] = (i * data.loc[temp["index"][i-1], TS] + (i+2) * data.loc[temp["index"][i+1], TS]) / (2*(i+1))

100%|██████████| 30/30 [00:58<00:00,  1.95s/it]


In [ ]:
data.to_csv("preprocess_3.csv", index = False)